In [1]:
import pandas as pd
import folium
from folium import PolyLine

In [2]:
atrai_bike_data = pd.read_csv ('combined_data_21_01_2025.csv')

# Group by 'device_id' and count the number of rows for each device
device_counts = atrai_bike_data.groupby('device_id').size()

# Filter out device_ids with fewer than 10 data entries
valid_device_ids = device_counts[device_counts >= 10].index

# Filter the original DataFrame to keep only rows with valid device_ids
atrai_bike_data = atrai_bike_data[atrai_bike_data['device_id'].isin(valid_device_ids)]

In [3]:
road_roughness = atrai_bike_data[['Surface Asphalt', 'Surface Sett', 'Surface Compacted', 'Surface Paving', 'lng', 'lat']].copy()
road_roughness = road_roughness.dropna(subset=['Surface Asphalt', 'Surface Sett', 'Surface Compacted', 'Surface Paving'])

# Roughness scores
roughness_scores = {
    'Asphalt': 1,
    'Paving': 2,
    'Compacted': 3,
    'Sett': 4
}

# Function to calculate roughness score
def calculate_roughness(row):
    score = 0
    score += roughness_scores['Asphalt'] * (row['Surface Asphalt'])
    score += roughness_scores['Paving'] * (row['Surface Paving'])
    score += roughness_scores['Compacted'] * (row['Surface Compacted'])
    score += roughness_scores['Sett'] * (row['Surface Sett'])
    return score

# Applying the function to each row to calculate the roughness score
road_roughness['Roughness'] = road_roughness.apply(calculate_roughness, axis=1)

# Normalizing the roughness
road_roughness['Roughness_Normalized'] = (road_roughness['Roughness'] / road_roughness['Roughness'].max()) * 100

In [4]:
# Assuming you have a DataFrame with lat, lng, and Roughness_Normalized (0-100)
road_roughness_clean = road_roughness.dropna(subset=['lat', 'lng', 'Roughness_Normalized'])

# Initialize a map centered on Münster
m_roughness = folium.Map(location=[51.9625, 7.6256], zoom_start=13)

# Color based on the roughness index
def get_color(roughness):
    if roughness <= 20:
        return 'green'
    elif roughness <= 40:
        return 'lightgreen'
    elif roughness <= 60:
        return 'yellow'
    elif roughness <= 80:
        return 'orange'
    else:
        return 'red'

# Group data by nearby road segments (if possible) or directly use points with connections (polylines)
# This example assumes the lat/lng pairs in the data are sequential road segments.
# For more complex road segments, you would need GeoJSON data with exact road shapes.
for idx, row in road_roughness_clean.iterrows():
    lat = row['lat']
    lng = row['lng']
    roughness = row['Roughness_Normalized']
    
    # Assign a color based on roughness value
    color = get_color(roughness)
    
    # You would use PolyLine here if you have continuous segments or just points.
    # Adding a PolyLine for simplicity here, but you could replace it with actual road data:
    folium.CircleMarker([lat, lng], radius=5, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(m_roughness)

# Add a legend to describe the roughness levels
legend_html = '''
<div style="position: fixed; bottom: 30px; left: 30px; width: 300px; height: 160px;
            background-color: white; z-index:9999; font-size:14px;
            border:2px solid grey; padding: 10px; text-align: center;">
    <b>Road Roughness Index (RRI)</b><br>
    
    <!-- Small color boxes above each range -->
    <div style="display: flex; justify-content: space-between; margin: 5px auto; width: 250px;">
        <div style="width: 20px; height: 20px; background: green;"></div>
        <div style="width: 20px; height: 20px; background: lightgreen;"></div>
        <div style="width: 20px; height: 20px; background: yellow;"></div>
        <div style="width: 20px; height: 20px; background: orange;"></div>
        <div style="width: 20px; height: 20px; background: red;"></div>
    </div>

    <!-- Labels for ranges -->
    <div style="display: flex; justify-content: space-between; margin: 5px auto; width: 250px;">
        <span>0-20</span>
        <span>21-40</span>
        <span>41-60</span>
        <span>61-80</span>
        <span>81-100</span>
    </div>
    
    <p style="margin-top: 10px; font-size: 12px; text-align: center;">
        RRI based on surface textures<br>
        calculated by senseBox:bike
    </p>
</div>
'''



# Add legend
legend = folium.Element(legend_html)
m_roughness.get_root().html.add_child(legend)

# Save the map as an HTML file
m_roughness.save("road_roughness_colored_map.html")